In [1]:
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from transformers import BartForConditionalGeneration, BartConfig
from transformers import Trainer, TrainingArguments, Seq2SeqTrainingArguments, Seq2SeqTrainer
from preprocessing import *
import os
from glob import glob
import re

2021-10-16 15:31:44.774942: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
preprocess = Preprocess()

In [3]:
train_path_list = glob('./Train/*')

In [4]:
train_json_list = preprocess.make_dataset_list(train_path_list)
train_data= preprocess.make_set_as_df(train_json_list)
encoder_input_train, decoder_output_train = preprocess.make_tokenizer_input(train_data)

In [5]:
print(encoder_input_train[:2])
print(encoder_input_train[-2:])

['근데 왜 막말을 할까... 이유가 뭘까... 남의 기분따위 상관없다는 그런 뉘앙스는 뭘까.. 허..빡쳐 흥분하면 감정적이여 지자나,, 이걸 잘 다스리는 사람이 진짜 멋지더라,, 근데 그걸 조절을 해야지 나이가 맣ㄴ으면 긍깨,,,', '대박 #@소속# 다니던 #@이름# 아는사람 ㅋㅋㅋ옹아로 하얀애 방금 이쁘장한애 생생정보통 나옴 ㅋㅋㅋㅋㅋ 잉 왜??? 왜?? 족발 무한리필집 나오는데 인천에 잇는거고든 걔 거기서 먹고 인터뷰함']
[' [CLS] 집에서 밥을 해먹어야 해서 엄마 카드(엄카)를 가지고 장을 보러 간다. [SEP] ', ' [CLS] 실바니안은 코스트코에 가서 사는 것보다 인터넷으로 사는 게 더 싸다. [SEP] ']


# 전처리

In [6]:
from soynlp.normalizer import *
#from pykospacing import Spacing
from tqdm.auto import tqdm

def delete_char(texts):
    preprocessed_text = []
    proc = re.compile(r"[^ㄱ-ㅎ가-힣a-zA-Z!?@#$%^&*()_ +]")
    for text in tqdm(texts):
        text = proc.sub("", text).strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

def spacing_sent(texts):
    """
    띄어쓰기를 보정합니다.
    """
    spacing = Spacing()
    preprocessed_text = []
    for text in tqdm(texts):
        text = spacing(text)
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

def remove_repeat_char(texts):
    preprocessed_text = []
    for text in tqdm(texts):
        text = repeat_normalize(text, num_repeats=2).strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [7]:
encoder_input_train = delete_char(encoder_input_train)
encoder_input_train = remove_repeat_char(encoder_input_train)
#encoder_input_train = spacing_sent(encoder_input_train)

In [8]:
print(encoder_input_train[:2])
print(encoder_input_train[-2:])

['근데 왜 막말을 할까 이유가 뭘까 남의 기분따위 상관없다는 그런 뉘앙스는 뭘까 허빡쳐 흥분하면 감정적이여 지자나 이걸 잘 다스리는 사람이 진짜 멋지더라 근데 그걸 조절을 해야지 나이가 맣ㄴ으면 긍깨', '대박 #@소속# 다니던 #@이름# 아는사람 ㅋㅋㅋ옹아로 하얀애 방금 이쁘장한애 생생정보통 나옴 ㅋㅋ 잉 왜??? 왜?? 족발 무한리필집 나오는데 인천에 잇는거고든 걔 거기서 먹고 인터뷰함']
['CLS 집에서 밥을 해먹어야 해서 엄마 카드(엄카)를 가지고 장을 보러 간다 SEP', 'CLS 실바니안은 코스트코에 가서 사는 것보다 인터넷으로 사는 게 더 싸다 SEP']


# Make Tokenizer

In [9]:
from tokenizers import BertWordPieceTokenizer 

In [10]:
vocab_dir = './vocab/vocab_preprocessed.txt'

In [11]:
print('-'*10, 'Make tokenizer and train', '-'*10,)
tokenizer = BertWordPieceTokenizer(
    None,
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False, # Must be False if cased model
    lowercase=False,
    wordpieces_prefix="##",
)
tokenizer.add_special_tokens(["[PAD]", "[CLS]", "[UNK]", "[SEP]", "[MASK]"]) #"<pad>", "<s>", "</s>", "<sep>", "<mask>"
tokenizer.train_from_iterator(
    encoder_input_train,
    vocab_size=50000,
    min_frequency=2,
    show_progress=True,
    special_tokens = ["[PAD]", "[CLS]", "[UNK]", "[SEP]", "[MASK]"],#"<pad>", "<s>", "</s>", "<sep>", "<mask>"
    wordpieces_prefix="##",
)
print('-'*10, 'Make tokenizer and train complete', '-'*10,)

---------- Make tokenizer and train ----------



---------- Make tokenizer and train complete ----------


In [12]:
vocab = tokenizer.get_vocab()
print(len(vocab))

vocabulary = [[v, k] for k, v in vocab.items()]
vocabulary = sorted(vocabulary)
vocabulary = list(np.array(vocabulary)[:, 1])

with open(vocab_dir, 'w+') as lf:
    lf.write('\n'.join(vocabulary))

50000


# Load Tokenizer

In [52]:
from transformers import BertTokenizer

In [54]:
bert_tokenizer = BertTokenizer.from_pretrained('./vocab/', do_basic_tokenize=False) #(vocab_file = './vocab/vocab.txt',  do_basic_tokenize=False)

Didn't find file ./vocab/added_tokens.json. We won't load it.
Didn't find file ./vocab/special_tokens_map.json. We won't load it.
Didn't find file ./vocab/tokenizer_config.json. We won't load it.
Didn't find file ./vocab/tokenizer.json. We won't load it.
loading file ./vocab/vocab.txt
loading file None
loading file None
loading file None
loading file None
file ./vocab/config.json not found


# Model Pretraining

In [55]:
from transformers import BartForCausalLM, BartConfig, BartForConditionalGeneration
from transformers.tokenization_utils import PreTrainedTokenizer
from transformers import DataCollatorForLanguageModeling, DataCollatorForPermutationLanguageModeling
import torch

In [56]:
class LineByLineTextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(self, tokenizer: PreTrainedTokenizer, data, block_size: int):
        batch_encoding = tokenizer(data, add_special_tokens=True, truncation=True, max_length=block_size)
        self.examples = batch_encoding["input_ids"]
        self.examples = [{"input_ids": torch.tensor(e, dtype=torch.long)} for e in self.examples]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

In [58]:
dataset = LineByLineTextDataset(
        tokenizer=bert_tokenizer,
        data=list(encoder_input_train),
        block_size=256,
    )

In [59]:
# set mlm task  DataCollatorForSOP(DataCollatorForLanguageModeling)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=bert_tokenizer, mlm=True, mlm_probability=0.15
)
# data_collator = DataCollatorForPermutationLanguageModeling(
#     tokenizer=bert_tokenizer, max_span_length=5, plm_probability=0.15
# )

In [60]:
config = BartConfig()
config.d_model = 1024
config.encoder_ffn_dim = 512
config.decoder_ffn_dim = 512
config.encoder_attention_heads = 8
config.decoder_start_token_id = 8
config.max_position_embeddings = 512
config.encoder_layers = 6
config.decoder_layers = 6
model = BartForCausalLM(config=config)
model.resize_token_embeddings(len(bert_tokenizer))

Embedding(50000, 1024)

In [21]:
# set training args
training_args = Seq2SeqTrainingArguments(
    output_dir='./outputs_preprocessing/',
    overwrite_output_dir=True,
    num_train_epochs=15,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=10,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    save_total_limit=1,
    load_best_model_at_end=True,
    seed=42,
)
# set Trainer class for pre-training
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    eval_dataset=dataset,    
)

In [33]:
trainer.train()

***** Running training *****
  Num examples = 232062
  Num Epochs = 15
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 320
  Gradient Accumulation steps = 10
  Total optimization steps = 10875


Epoch,Training Loss,Validation Loss
0,8.941600,8.369273
1,8.496500,8.241735
2,8.330000,8.174495
3,8.299200,8.101591
4,8.228400,8.054579
5,8.203600,8.003556
6,8.143200,7.945886
7,8.109300,7.886611
8,8.039200,7.834373
9,8.015800,7.803403


***** Running Evaluation *****
  Num examples = 232062
  Batch size = 8
Saving model checkpoint to ./outputs_SOP/checkpoint-725
Configuration saved in ./outputs_SOP/checkpoint-725/config.json
Model weights saved in ./outputs_SOP/checkpoint-725/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 232062
  Batch size = 8
Saving model checkpoint to ./outputs_SOP/checkpoint-1450
Configuration saved in ./outputs_SOP/checkpoint-1450/config.json
Model weights saved in ./outputs_SOP/checkpoint-1450/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 232062
  Batch size = 8
Saving model checkpoint to ./outputs_SOP/checkpoint-2175
Configuration saved in ./outputs_SOP/checkpoint-2175/config.json
Model weights saved in ./outputs_SOP/checkpoint-2175/pytorch_model.bin
Deleting older checkpoint [outputs_SOP/checkpoint-725] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 232062
  Batch size = 8
Saving model checkpoint to ./outputs_SOP/checkpoint-

TrainOutput(global_step=10875, training_loss=8.147608117816091, metrics={'train_runtime': 11873.9712, 'train_samples_per_second': 293.156, 'train_steps_per_second': 0.916, 'total_flos': 2.6954582842773504e+16, 'train_loss': 8.147608117816091, 'epoch': 15.0})

# Model fine tuning

In [34]:
# input_ids=None,
# attention_mask=None,
# decoder_input_ids=None,
# decoder_attention_mask=None,
# labels=None,

In [61]:
class Mydataset(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()}
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2)
        item['labels'] = self.labels['input_ids'][idx]
        return item
    
    def __len__(self):
        return self.len

In [62]:
del model

In [37]:
path_list = glob('./outputs_SOP/*')

In [63]:
config = BartConfig()
config.d_model = 1024
config.encoder_ffn_dim = 512
config.decoder_ffn_dim = 512
config.encoder_attention_heads = 8
config.decoder_start_token_id = 8
config.max_position_embeddings = 512
config.encoder_layers = 6
config.decoder_layers = 6
generate_model = BartForConditionalGeneration(config=config)
generate_model.resize_token_embeddings(len(bert_tokenizer))

Embedding(50000, 1024)

In [64]:
encoder_input_train, decoder_input_train, decoder_output_train = preprocess.make_model_input(train_data)

In [65]:
tokenized_encoder_inputs = bert_tokenizer(list(encoder_input_train), return_tensors="pt", padding=True, 
                            add_special_tokens=True, truncation=True, max_length=256, return_token_type_ids=False,)
tokenized_decoder_inputs = bert_tokenizer(list(decoder_input_train), return_tensors="pt", padding=True, 
                            add_special_tokens=True, truncation=True, max_length=50, return_token_type_ids=False,)
tokenized_decoder_ouputs = bert_tokenizer(list(decoder_output_train), return_tensors="pt", padding=True, 
                            add_special_tokens=True, truncation=True, max_length=50, return_token_type_ids=False,)

encoder_inputs_dataset = Mydataset(tokenized_encoder_inputs, tokenized_decoder_inputs, tokenized_decoder_ouputs, len(encoder_input_train))

In [66]:
# set training args
training_args = Seq2SeqTrainingArguments(
    output_dir='./bart_prerprocess_without_pretraining_v3/',
    overwrite_output_dir=True,
    num_train_epochs=50,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=10,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    save_total_limit=1,
    load_best_model_at_end=True,
    seed=42,
)
# set Trainer class for pre-training
trainer = Seq2SeqTrainer(
    model=generate_model,
    args=training_args,
    train_dataset=encoder_inputs_dataset,
    eval_dataset=encoder_inputs_dataset,    
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 116031
  Num Epochs = 50
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 320
  Gradient Accumulation steps = 10
  Total optimization steps = 18100


Epoch,Training Loss,Validation Loss
0,No log,1.883409
1,2.036200,1.673305
2,1.703800,1.505078
3,1.703800,1.380668


***** Running Evaluation *****
  Num examples = 116031
  Batch size = 32
Saving model checkpoint to ./bart_prerprocess_without_pretraining_v3/checkpoint-362
Configuration saved in ./bart_prerprocess_without_pretraining_v3/checkpoint-362/config.json
Model weights saved in ./bart_prerprocess_without_pretraining_v3/checkpoint-362/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 116031
  Batch size = 32
Saving model checkpoint to ./bart_prerprocess_without_pretraining_v3/checkpoint-724
Configuration saved in ./bart_prerprocess_without_pretraining_v3/checkpoint-724/config.json
Model weights saved in ./bart_prerprocess_without_pretraining_v3/checkpoint-724/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 116031
  Batch size = 32
Saving model checkpoint to ./bart_prerprocess_without_pretraining_v3/checkpoint-1086
Configuration saved in ./bart_prerprocess_without_pretraining_v3/checkpoint-1086/config.json
Model weights saved in ./bart_prerprocess_without_pretr

# Evaluation

In [1]:
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from transformers import BartForConditionalGeneration, BartConfig
from transformers import Trainer, TrainingArguments, Seq2SeqTrainingArguments, Seq2SeqTrainer
from preprocessing import *
import os
from glob import glob
import torch

In [4]:
preprocess = Preprocess()

In [44]:
generate_model = BartForConditionalGeneration.from_pretrained('./bart_prerprocess_without_pretraining_v2/checkpoint-18100/').to('cuda:0')
bert_tokenizer = BertTokenizer.from_pretrained('./vocab/', do_basic_tokenize=False)

loading configuration file ./bart_prerprocess_without_pretraining_v2/checkpoint-18100/config.json
Model config BartConfig {
  "_name_or_path": "./bart_prerprocess_without_pretraining/checkpoint-7240/",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 512,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 8,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 512,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "bart",

In [45]:
test_json_path = './Valid/미용과건강.json'
test_path_list = glob(test_json_path)
test_path_list.sort()

test_json_list = preprocess.make_dataset_list(test_path_list)
test_data = preprocess.make_set_as_df(test_json_list)

encoder_input_test, decoder_input_test = preprocess.make_model_input(test_data, is_test= True)

test_json_path :
./Valid/미용과건강.json
test_path_list :
['./Valid/미용과건강.json']
test_data:
0       따뜻하게 자라 요새 목은 좀 괜찮아 졌나 엉 괜찮아 수업을안하니까 엉 목에 수건감고...
1       머리 뽀글뽀글 말아? 머리위에만 옆은 보통 알아서 해준데 알아서 잘 해주겠지ㅎㅎ 남...
2       갑자기 엉뽕 #@기타# 극단 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 아니 엄마가 엉덩이 이야...
3       우리 식습관 고치자 진ㅉ 맵고 짠거 스탑 머리론 알겟다고 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 실천이 ...
4       일단..검사결과나올때까진 숨길라했는데 망쳤네 안돼ㅜㅜㅜ 내가알아야지ㅜㅡㅠㅜ 진짜가면...
                              ...                        
2054    병원 간겨?? 웅웅 조금 전에 막 도착. 붕대감은 거랑 테이프 떼었는데... 아직 ...
2055    ㅋㅋㅋㅋ허리는 많이 좋아졌어?? 웅! 막 예전엔 허리아팠는데 지금은 허리좀아픈애 좀...
2056    방금 엄마 스쿼트 알려드리고 왔어 갑자기 아빠가 체성분체중계가 있다고 꺼내오셨는데 ...
2057    그렇죠 특히나 기관지에 예민하신 분들은 맞아요 너무 힘들어 하시더라고요 저희 사장님...
2058    아 나 파마한거 다 풀림;; 왼쪽 거의다 풀림 짜증낰ㅋㅋㅋ 헉 미친 벌써..? 사진...
Name: dialogue, Length: 2059, dtype: object


In [46]:
encoder_input_test = delete_char(encoder_input_test)
encoder_input_test = remove_repeat_char(encoder_input_test)

In [ ]:
print(encoder_input_test[:10])

In [47]:
si = 0
ei = 20
tokenized_encoder_inputs = bert_tokenizer(list(encoder_input_test[si:ei]), return_tensors="pt", padding=True, add_special_tokens=True, truncation=True, max_length=256, return_token_type_ids=False,)
with torch.no_grad():
    generated_ids = generate_model.generate(input_ids=tokenized_encoder_inputs['input_ids'].to('cuda:0'), max_length=100, num_beams=3)

In [48]:
summary = [bert_tokenizer.decode(g_ids, skip_special_tokens=True) for g_ids in generated_ids.cpu().numpy()]

In [ ]:
tokenized = [bert_tokenizer.decode(g_ids) for g_ids in tokenized_encoder_inputs['input_ids']]

In [49]:
for d, tk, s, gt in zip(encoder_input_test[si:ei], tokenized, summary, test_data["summary"][si:ei]):
    print('-'*20)
    print('D:\t', d)
    print('TK:\t', tk)
    print('S:\t', s)
    print('gt:\t', gt)
    print('-'*20)

--------------------
D:	 따뜻하게 자라 요새 목은 좀 괜찮아 졌나 엉 괜찮아 수업을안하니까 엉 목에 수건감고자 젖은 수건 널어놔 알겠어 요샌느 별러 안건조해
S:	 '
gt:	 목이 건조하면 수건을 감고 자거나 젖은 수건을 널어놓으라고 한다.
--------------------
--------------------
D:	 머리 뽀글뽀글 말아? 머리위에만 옆은 보통 알아서 해준데 알아서 잘 해주겠지ㅎㅎ 남자가 해줘? 예쁜게해준데 응응ㅎㅎ 어 부원장 예쁘게 하고 와ㅎ 다하고 사진 찍어봐봐ㅎㅎ 영양제 미리바르고 해준다고 하더데
S:	 ' 머리 안
gt:	 머리를 어떤 스타일로 하는지 물어보고 예쁘게 하고 다 하고 사진 찍어 보내라고 한다.
--------------------
--------------------
D:	 갑자기 엉뽕 #@기타# 극단 ㅋㅋ 아니 엄마가 엉덩이 이야기햇어 엉덩이납닥하면 건강에도 안좋다 어쩌고저쩌고 옷입을때 어쩌고저쩌고 근데진짜 나 치마입으면 엉덩이 티 개많이나거둥 근데 엉뽕 ㅈㄹ티나고 좀 더 뚱뚱해보일거같애 차라리 플레어?치마 입는게 나을즛 금욜에 너 궁디봐여지 달라붙는거 아니면 티 안났던듯?? 금욜애 한번 봐보자 우리 옛날에 중딩때 가슴커보일라고 일부애들 가슴에 휴지넣고 양말넣은거 생각난다 ㅋㅋ나도 기억나
S:	 ' 엄마가 일이 너무 안 맞아서 살 것이 안 맞아서 옷 입으면
gt:	 엄마가 엉덩이가 납작하면 건강에도 좋지 않고 옷 입을 때도 예쁘지 않다고 말씀하셨으며 스스로도 치마를 입으면 엉덩이가 납작한 게 너무 티 나서 엉덩이 뽕(엉뽕)에 대해 이야기하고 있다.
--------------------
--------------------
D:	 우리 식습관 고치자 진ㅉ 맵고 짠거 스탑 머리론 알겟다고 ㅋㅋ 실천이 안돼서 그렇다니깐 그래도 무섭자나 영영 이러케 먹고 살 순 없어 새해부터 시작하자 올바른 식습관 만들기 프로젝트 ㅋㅋㅋ 진짜 맘 먹고 해보자 살도 빠지고 좋지 오케이 가즈아
S:	 ' 식습관이 먹고
gt

In [25]:
# TEST
index = 25
print(encoder_input_train[index])
print(bert_tokenizer.tokenize(encoder_input_train[index]))
print(bert_tokenizer.encode(encoder_input_train[index]))

ids = tokenizer.encode(encoder_input_train[index]).ids
print(ids)
a = [tokenizer.id_to_token(i) for i in ids]
print(a)

그리고 회사단톡 주말이건 저녁이건 계속 오는데 아니 왜 업무외시간에 그런대 먹는거 사진 올라옴 족발 찍어올리고 배고파짐 일상을 공유하고 난리야 먹방 동아리 같아
['그리고', '회사', '##단톡', '주말이', '##건', '저녁이', '##건', '계속', '오는데', '아니', '왜', '업무', '##외', '##시간에', '그런대', '먹는거', '사진', '올라옴', '족발', '찍어', '##올리고', '배고파', '##짐', '일상을', '공유하고', '난리야', '먹방', '동아리', '같아']
[1, 2325, 2385, 19550, 15296, 1088, 26846, 1088, 2122, 6204, 1998, 672, 9773, 1245, 4089, 33576, 5310, 2001, 20603, 9721, 3097, 23423, 3223, 1654, 33231, 10876, 13291, 14106, 10887, 2284, 3]
[2325, 2385, 19550, 15296, 1088, 26846, 1088, 2122, 6204, 1998, 672, 9773, 1245, 4089, 33576, 5310, 2001, 20603, 9721, 3097, 23423, 3223, 1654, 33231, 10876, 13291, 14106, 10887, 2284]
['그리고', '회사', '##단톡', '주말이', '##건', '저녁이', '##건', '계속', '오는데', '아니', '왜', '업무', '##외', '##시간에', '그런대', '먹는거', '사진', '올라옴', '족발', '찍어', '##올리고', '배고파', '##짐', '일상을', '공유하고', '난리야', '먹방', '동아리', '같아']
